<a href="https://colab.research.google.com/github/jiin124/2021_2_SAI_ML/blob/main/4%EC%A3%BC%EC%B0%A8/%EC%8A%A4%ED%83%9C%EA%B9%85_%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

df=load_breast_cancer()

X_data=df.data
y_label=df.target

X_train,X_test,y_train,y_test=train_test_split(X_data,y_label,test_size=0.2,random_state=0)


KNN,ADABoost,랜덤포레스트,결정트리를 이용해서 알고리즘 클래스를 형성하고 최종모델 로지스틱 회귀이다.

In [30]:
#개별 ML모델 생성
knn_clf=KNeighborsClassifier(n_neighbors=4)
rf_clf=RandomForestClassifier(n_estimators=100,random_state=0)
dt_clf=DecisionTreeClassifier()
ada_clf=AdaBoostClassifier(n_estimators=100)

lr_final=LogisticRegression(C=10)

#개별 모델 학습
knn_clf.fit(X_train,y_train)
rf_clf.fit(X_train,y_train)
dt_clf.fit(X_train,y_train)
ada_clf.fit(X_train,y_train)



AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=100, random_state=None)

In [31]:
knn_pred=knn_clf.predict(X_test)
rf_pred=rf_clf.predict(X_test)
dt_pred=dt_clf.predict(X_test)
ada_pred=ada_clf.predict(X_test)

print("KNN 정확도 : {0:.4f}".format(accuracy_score(y_test,knn_pred)))
print("랜덤 포레스트 : {0:.4f}".format(accuracy_score(y_test,rf_pred)))
print("결정 트리 정확도 : {0:.4f}".format(accuracy_score(y_test,dt_pred)))
print("에이다부스트 정확도 : {0:.4f}".format(accuracy_score(y_test,ada_pred)))

KNN 정확도 : 0.9211
랜덤 포레스트 : 0.9649
결정 트리 정확도 : 0.9035
에이다부스트 정확도 : 0.9561


개별 알고리즘으로부터 예측된 예측값을 칼럼 레벨로 옆으로 붙여서 피처값을 만들어 최종 메타 모델인 로지스틱 회귀에서 학습 데이터로 다시 사용하겠다. 
반환된 예측 데이터는 1차원 형태의 ndarray이니까 먼저 반환된 예측 결과를 행 형태로 붙인 뒤 넘파이의 tranpose()를 이용해 행과 열 위치를 바꾼 ndarray로 변환하면 된다. 

In [32]:
pred=np.array([knn_pred,rf_pred,dt_pred,ada_pred])
print(pred.shape)

#tranpose를 이용해 행과 열 위치 교환 칼럼 레벨로 각 알고리즘의 예측 결과를 피처로 만듦
pred=np.transpose(pred)
print(pred.shape)

(4, 114)
(114, 4)


이렇게 예측 데이터로 생성된 데이터 세트를 기반으로 최종 메타 모델인 로지스틱 회귀를 학습하고 예측 정확도를 측정하겠다. 

In [33]:
lr_final.fit(pred,y_test)
final=lr_final.predict(pred)

print("최종 메타 모델의 예측 정확도 :{0:.4f}".format(accuracy_score(y_test,final)))

최종 메타 모델의 예측 정확도 :0.9649


# CV세트 기반의 스태킹

## step1

In [38]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

#개별 기반 모델에서 최종 메타 모델이 사용할 학습 및 테스트용 데이터를 생성하기 위한 함수
def get_stacking_base_datsets(model,X_train_n,y_train_n,X_test_n,n_folds):
  #지정된 n_folds값으로 KFold 생성.
  kf=KFold(n_splits=n_folds,shuffle=False,random_state=0)
  #추후에 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화
  train_fold_pred=np.zeros((X_train_n.shape[0],1))
  test_pred=np.zeros((X_test_n.shape[0],n_folds))
  print(model.__class__.__name__,'모델 시작')

  for folder_counter,(train_index,valid_index) in enumerate(kf.split(X_train_n)):
    #입력된 학습 데이터에서 기반 모델이 학습.예측할 폴드 데이터 추출
    print("\t 폴드 세트 : ",folder_counter,'시작')
    X_tr=X_train_n[train_index]
    y_tr=y_train_n[train_index]
    X_te=X_train_n[valid_index]

    #폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행
    model.fit(X_tr,y_tr)
    #폴드 세트 내부에서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장.
    train_fold_pred[valid_index,:]=model.predict(X_te).reshape(-1,1)
    #입력된 원본 테스트 데이터를 폴드 세트내 학습된 기반 모델에서 예측 후 데이터 저장.
    test_pred[:,folder_counter]=model.predict(X_test_n)

  #폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균해 테스트 데이터로 생성
  test_pred_mean=np.mean(test_pred,axis=1).reshape(-1,1)

  #train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
  return train_fold_pred,test_pred_mean

In [39]:
knn_train,knn_test=get_stacking_base_datsets(knn_clf,X_train,y_train,X_test,7)
rf_train,rf_test=get_stacking_base_datsets(rf_clf,X_train,y_train,X_test,7)
dt_train,dt_test=get_stacking_base_datsets(dt_clf,X_train,y_train,X_test,7)
ada_train,ada_test=get_stacking_base_datsets(ada_clf,X_train,y_train,X_test,7)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


KNeighborsClassifier 모델 시작
	 폴드 세트 :  0 시작
	 폴드 세트 :  1 시작
	 폴드 세트 :  2 시작
	 폴드 세트 :  3 시작
	 폴드 세트 :  4 시작
	 폴드 세트 :  5 시작
	 폴드 세트 :  6 시작
RandomForestClassifier 모델 시작
	 폴드 세트 :  0 시작
	 폴드 세트 :  1 시작
	 폴드 세트 :  2 시작
	 폴드 세트 :  3 시작
	 폴드 세트 :  4 시작
	 폴드 세트 :  5 시작
	 폴드 세트 :  6 시작


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


DecisionTreeClassifier 모델 시작
	 폴드 세트 :  0 시작
	 폴드 세트 :  1 시작
	 폴드 세트 :  2 시작
	 폴드 세트 :  3 시작
	 폴드 세트 :  4 시작
	 폴드 세트 :  5 시작
	 폴드 세트 :  6 시작
AdaBoostClassifier 모델 시작
	 폴드 세트 :  0 시작
	 폴드 세트 :  1 시작
	 폴드 세트 :  2 시작
	 폴드 세트 :  3 시작
	 폴드 세트 :  4 시작
	 폴드 세트 :  5 시작
	 폴드 세트 :  6 시작


## step2

앞의 예제에서 get_stacking_base_datasets()호출로 반환된 각 모델별 학습 데이터와 테스트 데이터를 합치기만 하면 된다. 넘파이의 concatenate()를 이용해 쉽게 이와 같은 기능을 수행한다. concatenate()는 여러 개의 넘파이 배열을 이용해 쉽게 이와 같은 기능을 수행한다. concatenate()는 여러개의 넘파이 배열을 칼럼 또는 로우 레벨로 합쳐주는 기능을 제공한다. 

In [40]:
Stack_final_X_train=np.concatenate((knn_train,rf_train,dt_train,ada_train),axis=1)
Stack_final_X_test=np.concatenate((knn_test,rf_test,dt_test,ada_test),axis=1)
print("원본 학습 피처 데이터 shape:",X_train.shape,'원본 테스트 피처 shape:',X_test.shape)
print('스태킹 학습 피처 데이터 shape:',Stack_final_X_train.shape,
      '스태킹 테스트 피처 데이터 shape:',Stack_final_X_test.shape)


원본 학습 피처 데이터 shape: (455, 30) 원본 테스트 피처 shape: (114, 30)
스태킹 학습 피처 데이터 shape: (455, 4) 스태킹 테스트 피처 데이터 shape: (114, 4)


In [41]:
lr_final.fit(Stack_final_X_train,y_train)
stack_final=lr_final.predict(Stack_final_X_test)

print('최종 메타 모델의 예측 정확도:{0:.4f}'.format(accuracy_score(y_test,stack_final)))

최종 메타 모델의 예측 정확도:0.9737
